In [300]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from decouple import config
import decouple
config = decouple.AutoConfig('g:\PythonPractice\python practice.py\guvi\projects\Local Food Wastage Management System')



<>:7: SyntaxWarning: invalid escape sequence '\P'
<>:7: SyntaxWarning: invalid escape sequence '\P'
C:\Users\OWNER\AppData\Local\Temp\ipykernel_9112\3924297067.py:7: SyntaxWarning: invalid escape sequence '\P'
  config = decouple.AutoConfig('g:\PythonPractice\python practice.py\guvi\projects\Local Food Wastage Management System')


Data preprocessing 

In [301]:
data_claims = pd.read_csv('data/claims_data.csv')
data_food_listings = pd.read_csv('data/food_listings_data.csv')
data_providers = pd.read_csv('data/providers_data.csv')
data_receivers = pd.read_csv('data/receivers_data.csv')

In [302]:
data_receivers.head()
data_receivers.info()
data_receivers.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Receiver_ID  1000 non-null   int64 
 1   Name         1000 non-null   object
 2   Type         1000 non-null   object
 3   City         1000 non-null   object
 4   Contact      1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


,Receiver_ID
count,1000.000000
mean,500.500000
std,288.819436
min,1.000000
25%,250.750000
50%,500.500000
75%,750.250000
max,1000.000000


In [303]:
data_providers.head()
data_providers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Provider_ID  1000 non-null   int64 
 1   Name         1000 non-null   object
 2   Type         1000 non-null   object
 3   Address      1000 non-null   object
 4   City         1000 non-null   object
 5   Contact      1000 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB


In [304]:
data_food_listings.head()
# data_food_listings['Food_Type'].value_counts()
data_food_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Food_ID        1000 non-null   int64 
 1   Food_Name      1000 non-null   object
 2   Quantity       1000 non-null   int64 
 3   Expiry_Date    1000 non-null   object
 4   Provider_ID    1000 non-null   int64 
 5   Provider_Type  1000 non-null   object
 6   Location       1000 non-null   object
 7   Food_Type      1000 non-null   object
 8   Meal_Type      1000 non-null   object
dtypes: int64(3), object(6)
memory usage: 70.4+ KB


In [305]:
data_claims.head()
data_claims.info()
data_claims.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Claim_ID     1000 non-null   int64 
 1   Food_ID      1000 non-null   int64 
 2   Receiver_ID  1000 non-null   int64 
 3   Status       1000 non-null   object
 4   Timestamp    1000 non-null   object
dtypes: int64(3), object(2)
memory usage: 39.2+ KB


,Claim_ID,Food_ID,Receiver_ID
count,1000.000000,1000.000000,1000.000000
mean,500.500000,502.163000,492.437000
std,288.819436,292.462255,284.487869
min,1.000000,1.000000,1.000000
25%,250.750000,233.750000,256.000000
50%,500.500000,504.500000,480.500000
75%,750.250000,762.000000,742.250000
max,1000.000000,999.000000,1000.000000


Null and duplicate remove


In [306]:
data_claims.isnull().sum()    #None
data_claims.duplicated().sum()    #None


np.int64(0)

In [307]:
data_food_listings.isnull().sum()   #None
data_food_listings.duplicated().sum()   #None

np.int64(0)

In [308]:
data_providers.isnull().sum()   #None
data_providers.duplicated().sum()   #None

np.int64(0)

In [309]:
data_receivers.isnull().sum()   #None
data_receivers.duplicated().sum()   #None

np.int64(0)

Mysql connection

In [310]:
import MySQLdb

try:
    db = MySQLdb.connect(
        host=config('db_host'),
        user=config("db_user"),
        passwd=config('db_password'),
        port=int(config('db_port')),
        # database='food_wastage_management_system'
    )
    cursor = db.cursor()
    print("Connected to the database successfully!")
except MySQLdb.Error as e:
    print(f"Error connecting to database: {e}")



Connected to the database successfully!


create databases

In [311]:
cursor.execute("CREATE DATABASE IF NOT EXISTS food_wastage_management_system")
db.commit()

In [312]:
query_use_db = "USE food_wastage_management_system"
cursor.execute(query_use_db)

# Creating  Table

cursor.execute("""
    CREATE TABLE IF NOT EXISTS providers (
        Provider_ID INT PRIMARY KEY,
        Name VARCHAR(255),
        Type VARCHAR(100),
        Address TEXT,
        City VARCHAR(100),
        Contact VARCHAR(50)
    )
""")
cursor.execute("""
    CREATE TABLE IF NOT EXISTS receivers (
        Receiver_ID INT PRIMARY KEY,
        Name VARCHAR(255),
        Type VARCHAR(100),
        City VARCHAR(100),
        Contact VARCHAR(50)
    )
""")
cursor.execute("""
    CREATE TABLE IF NOT EXISTS food_listings (
        Food_ID INT PRIMARY KEY,
        Food_Name VARCHAR(255),
        Quantity INT,
        Expiry_Date DATE,       
        Provider_ID INT,
        Provider_Type VARCHAR(100),
        Location VARCHAR(255),
        Food_Type VARCHAR(100),
        Meal_Type VARCHAR(100),          
        FOREIGN KEY (Provider_ID) REFERENCES providers(Provider_ID)
    )
""")
cursor.execute("""
    CREATE TABLE IF NOT EXISTS claims (
        Claim_ID INT PRIMARY KEY,
        Food_ID INT,
        Receiver_ID INT,
        Status VARCHAR(50),
        Timestamp DATETIME,       
        FOREIGN KEY (Food_ID) REFERENCES food_listings(Food_ID),
        FOREIGN KEY (Receiver_ID) REFERENCES receivers(Receiver_ID)
    )
""")
db.commit()

In [313]:
# data_food_listings['Expiry_Date']
# data_providers['Provider_ID'].values

In [314]:
# data_claims['Timestamp']

In [315]:
data_food_listings['Expiry_Date'] = pd.to_datetime(data_food_listings['Expiry_Date'], format='%m/%d/%Y')
data_claims['Timestamp'] = pd.to_datetime(
    data_claims['Timestamp'], 
    format='%m/%d/%Y %H:%M'
)


In [316]:
#Insert data into tables
for _, row in data_providers.iterrows():
    cursor.execute("""
        INSERT INTO providers (Provider_ID, Name, Type, Address, City, Contact)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, tuple(row))
for _, row in data_receivers.iterrows():
    cursor.execute("""
        INSERT INTO receivers (Receiver_ID, Name, Type, City, Contact)
        VALUES (%s, %s, %s, %s, %s)
    """, tuple(row))
for _, row in data_food_listings.iterrows():
    cursor.execute("""
        INSERT INTO food_listings (Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, tuple(row))
for _, row in data_claims.iterrows():
    cursor.execute("""
        INSERT INTO claims (Claim_ID, Food_ID, Receiver_ID, Status, Timestamp)
        VALUES (%s, %s, %s, %s, %s)
    """, tuple(row))      
cursor.execute("COMMIT")
cursor.close()



If Lock wait timeout exceeded; try restarting transaction

In [317]:
# cursor.execute('show processlist')
# processes = cursor.fetchall()
# for process in processes:
#     print(process)

In [318]:
# sleeping_ids = [24,25,26]  # the ones from SHOW PROCESSLIST
# for pid in sleeping_ids:
#     cursor.execute(f"kill {pid}")

Question 